<a href="https://colab.research.google.com/github/jinsyabeycg/BITS-AIML-Mtech-V2/blob/master/Group_136_NLP_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Group ID:
### Group Members Name with Student ID:
1. Nimishkumar Patel
2. Jinsy Abey
3. Sunil Pandey
4. Sufiya
5. Ashwani


In [54]:
# Constants and Imports

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import math
import matplotlib.pyplot as plt
import kagglehub
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
import re

## 1. Download the dataset and Create a dataframe named as News, then check the head, info, and describe methods on created dataframe News. (1 Mark)

In [55]:

# Download latest version
path = kagglehub.dataset_download("ankurzing/sentiment-analysis-for-financial-news")
print(path)

News = pd.read_csv('FinancialPhraseBank.csv', encoding='ISO-8859-1')
News.columns = ['Rating', 'Content']
head_output = News.head()
info_output = News.info()
describe_output = News.describe(include='all')

(head_output, info_output, describe_output)

/kaggle/input/sentiment-analysis-for-financial-news
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4845 entries, 0 to 4844
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Rating   4845 non-null   object
 1   Content  4845 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


(     Rating                                            Content
 0   neutral  Technopolis plans to develop in stages an area...
 1  negative  The international electronic industry company ...
 2  positive  With the new production plant the company woul...
 3  positive  According to the company 's updated strategy f...
 4  positive  FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,
 None,
          Rating                                            Content
 count      4845                                               4845
 unique        3                                               4837
 top     neutral  TELECOMWORLDWIRE-7 April 2006-TJ Group Plc sel...
 freq       2878                                                  2)

## 2. Perform pre-processing steps like Removing Punctuations, Numbers, and Special Characters, Stop Words in dataset. (1 Mark)

In [56]:
# Define stop words
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

nltk.download('stopwords')
print(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuations, numbers, special characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Remove stop words
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply preprocessing to the 'Content' column
News['Cleaned_Headline'] = News['Content'].apply(preprocess_text)

News[['Rating', 'Content', 'Cleaned_Headline']].head()

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Rating,Content,Cleaned_Headline
0,neutral,Technopolis plans to develop in stages an area...,technopolis plans develop stages area less squ...
1,negative,The international electronic industry company ...,international electronic industry company elco...
2,positive,With the new production plant the company woul...,new production plant company would increase ca...
3,positive,According to the company 's updated strategy f...,according company updated strategy years baswa...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,financing aspocomp growth aspocomp aggressivel...


## 3. Normalize review by using Stemming or Lemmatization. (1 Mark)

In [57]:
# Lemmatization
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# Assuming 'Cleaned_Content' is the preprocessed column
News['Cleaned_Headline_Lemm'] = News['Cleaned_Headline'].apply(lemmatize_text)


# Stemming
stemmer = PorterStemmer()

def stem_text(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

News['Cleaned_Headline_Stemm'] = News['Cleaned_Headline'].apply(stem_text)

News[['Rating', 'Content', 'Cleaned_Headline', 'Cleaned_Headline_Lemm', 'Cleaned_Headline_Stemm']].head()


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Rating,Content,Cleaned_Headline,Cleaned_Headline_Lemm,Cleaned_Headline_Stemm
0,neutral,Technopolis plans to develop in stages an area...,technopolis plans develop stages area less squ...,technopolis plan develop stage area less squar...,technopoli plan develop stage area less squar ...
1,negative,The international electronic industry company ...,international electronic industry company elco...,international electronic industry company elco...,intern electron industri compani elcoteq laid ...
2,positive,With the new production plant the company woul...,new production plant company would increase ca...,new production plant company would increase ca...,new product plant compani would increas capac ...
3,positive,According to the company 's updated strategy f...,according company updated strategy years baswa...,according company updated strategy year baswar...,accord compani updat strategi year baswar targ...
4,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,financing aspocomp growth aspocomp aggressivel...,financing aspocomp growth aspocomp aggressivel...,financ aspocomp growth aspocomp aggress pursu ...


## 4. Preprocessed Newsheadline should be included in the News data frame as cleaned_headline. Plot word cloud for all classes of the tweets. (1 Mark)

## 5. Create two objects X and Y. X will be the cleaned_headline ' column of News data frame and Y will be the ' Sentiment' column. ( 5 Marks)

# 5. A. Create a Word2Vec object and split the data into training and testing sets. Train a Decision tree model. Display the confusion Matrix for both train and test.


# 5. B. Create a CBow object and split the data into training and testing sets. Train a Decision tree model. Display the confusion Matrix for both train and test.


# 5. C. Compare Word2Vec and CBoW. Answer without justification will not be awarded marks.

## 6. Display the HMM POS tagging on the first row of ‘cleaned_headline’. ( 1 Mark)